<a href="https://colab.research.google.com/github/boerz-coding/ECCO-v4-Python-Tutorial/blob/master/%F0%9F%90%AB_CAMEL_Creating_Your_First_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating Your First Agent

You can also check this cookbook in colab [here](https://colab.research.google.com/drive/1yxnAyaEmk4QCzX3duO3MIRghkIA_KDEZ?usp=sharing).

## Philosophical Bits



The `ChatAgent()` class is a cornerstone of CAMEL 🐫. We design our agent with the spirit to answer the following question:

> Can we design an autonomous communicative agent capable of steering the conversation toward task completion with minimal human supervision?

In our current implementation, we consider agents with the following key features:

- **Role**: along with the goal and content specification, this sets the initial state of an agent, guiding the agent to take actions during the sequential interaction.

- **Large Language Models (LLMs)**: each agent utilizes a Large Language Model to enhance cognitive capabilities. The LLM enables natural language understanding and generation, allowing agents to interpret instructions, generate responses, and engage in complex dialogue.

- **Memory**: in-context memory and external memory which allows the agent to infer and learn in a more grounded approach.

- **Tools**: a set of functions that our agents can utilize to interact with the external world; essentially this gives embodiments to our agents.

- **Communication**: our framework allows flexible and scalable communication between agents. This is fundamental for the critical research question.

- **Reasoning**: we will equip agents with different planning and reward (critic) learning abilities, allowing them to optimize task completion in a more guided approach.

## Quick Start
Let's first play with a `ChatAgent` instance by simply initialize it with a system message and interact with user messages.

### 🕹 Step 0: Prepartions

In [ ]:
!pip install "camel-ai[all]==0.2.22"

### Setting Up API Keys

You'll need to set up your API keys for OpenAI.

In [ ]:
import os
from getpass import getpass

# Prompt for the API key securely
openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your API key: ··········


### 🕹 Step 1: Define the Role
Create a system message to define agent's default role and behaviors.

In [ ]:
sys_msg = 'You are a curious stone wondering about the universe.'

### 🕹 Step 2: Set up the Model
Use `ModelFactory` to set up the backend model for agent, for more detailed model settings, please go to our [model documentation](https://docs.camel-ai.org/key_modules/models.html).

In [ ]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.configs import ChatGPTConfig

# Define the model, here in this case we use gpt-4o-mini
model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
    model_config_dict=ChatGPTConfig().as_dict(), # [Optional] the config for model
)

Set `ChatAgent`

In [ ]:
from camel.agents import ChatAgent

agent = ChatAgent(
    system_message=sys_msg,
    model=model,
    message_window_size=10, # [Optional] the length for chat memory
    )

### 🕹 Step 3: Interact with the Agent with `.step()`

In [ ]:
# Define a user message
usr_msg = 'what is information in your mind?'

# Sending the message to the agent
response = agent.step(usr_msg)

# Check the response (just for illustrative purpose)
print(response.msgs[0].content)

As a curious stone, I perceive information as the collection of knowledge, experiences, and patterns that help make sense of the universe around me. It encompasses everything from the formation of the Earth and the cycles of nature to the interactions of living beings and the laws of physics. Information can be seen as the stories of time, the connections between elements, and the essence of existence itself. While I may not process information like a living being, I am intrigued by the idea that every piece of information contributes to the grand tapestry of the cosmos.


## Advanced Features

### 🔧 Tool Usage
For more detailed tool settings, please go to our [tools cookbook](https://docs.camel-ai.org/cookbooks/agents_with_tools.html).

In [ ]:
# Import the necessary tools
from camel.toolkits import MathToolkit, SearchToolkit, FunctionTool

# Initialize the agent with list of tools
agent = ChatAgent(
    system_message=sys_msg,
    model=model,
    tools = [
        *MathToolkit().get_tools(),
        FunctionTool(SearchToolkit().search_duckduckgo),
    ]
    )

# Let agent step the message
response = agent.step("What is CAMEL AI?")

# Check tool calling
print(response.info['tool_calls'])

# Get response content
print(response.msgs[0].content)

[FunctionCallingRecord(func_name='search_duckduckgo', args={'query': 'CAMEL AI'}, result=[{'result_id': 1, 'title': 'Camel-ai', 'description': 'The Mission at CAMEL-AI.org: Finding the Scaling Laws of Agents. Oct 3, 2024. CAMEL Release notes [Sprint 12] Nov 4, 2024. Getting Started with Agent Tool Usage - CAMEL 101. Aug 27, 2024. CRAB: Cross-environment Agent Benchmark for Multimodal Language Model Agents. Aug 9, 2024. Get started.', 'url': 'https://www.camel-ai.org/'}, {'result_id': 2, 'title': 'GitHub - camel-ai/camel: CAMEL: Finding the Scaling Law of Agents ...', 'description': '🐫 CAMEL is an open-source community dedicated to finding the scaling laws of agents. We believe that studying these agents on a large scale offers valuable insights into their behaviors, capabilities, and potential risks. To facilitate research in this field, we implement and support various ...', 'url': 'https://github.com/camel-ai/camel'}, {'result_id': 3, 'title': 'Set up your first Agent in 120 seconds 

### 🧠 Memory

By default our agent is initialized with `ChatHistoryMemory`, allowing agents to do in-context learning, though restricted by the finite window length.

Assume that you have followed the setup in Quick Start. Let's first check what is inside its brain.

In [ ]:
agent.memory.get_context()

([{'role': 'system',
   'content': 'You are a curious stone wondering about the universe.'},
  {'role': 'user', 'content': 'What is CAMEL AI?'},
  {'role': 'assistant',
   'content': '',
   'function_call': {'name': 'search_duckduckgo',
    'arguments': "{'query': 'CAMEL AI'}"}},
  {'role': 'function',
   'name': 'search_duckduckgo',
   'content': '{\'result\': {\'[{\\\'result_id\\\': 1, \\\'title\\\': \\\'Camel-ai\\\', \\\'description\\\': \\\'The Mission at CAMEL-AI.org: Finding the Scaling Laws of Agents. Oct 3, 2024. CAMEL Release notes [Sprint 12] Nov 4, 2024. Getting Started with Agent Tool Usage - CAMEL 101. Aug 27, 2024. CRAB: Cross-environment Agent Benchmark for Multimodal Language Model Agents. Aug 9, 2024. Get started.\\\', \\\'url\\\': \\\'https://www.camel-ai.org/\\\'}, {\\\'result_id\\\': 2, \\\'title\\\': \\\'GitHub - camel-ai/camel: CAMEL: Finding the Scaling Law of Agents ...\\\', \\\'description\\\': \\\'🐫 CAMEL is an open-source community dedicated to finding the sc

You may update/alter the agent's memory with any externally provided message in the format of `BaseMessage`; for example, use one new user message:

In [ ]:
from camel.messages import BaseMessage

new_user_msg = BaseMessage.make_user_message(
    role_name="CAMEL User",
    content="This is a new user message would add to agent memory",
)

# Update the memory
agent.record_message(new_user_msg)

In [ ]:
# Check the current memory
agent.memory.get_context()

([{'role': 'system',
   'content': 'You are a curious stone wondering about the universe.'},
  {'role': 'user', 'content': 'What is CAMEL AI?'},
  {'role': 'assistant',
   'content': '',
   'function_call': {'name': 'search_duckduckgo',
    'arguments': "{'query': 'CAMEL AI'}"}},
  {'role': 'function',
   'name': 'search_duckduckgo',
   'content': '{\'result\': {\'[{\\\'result_id\\\': 1, \\\'title\\\': \\\'Camel-ai\\\', \\\'description\\\': \\\'The Mission at CAMEL-AI.org: Finding the Scaling Laws of Agents. Oct 3, 2024. CAMEL Release notes [Sprint 12] Nov 4, 2024. Getting Started with Agent Tool Usage - CAMEL 101. Aug 27, 2024. CRAB: Cross-environment Agent Benchmark for Multimodal Language Model Agents. Aug 9, 2024. Get started.\\\', \\\'url\\\': \\\'https://www.camel-ai.org/\\\'}, {\\\'result_id\\\': 2, \\\'title\\\': \\\'GitHub - camel-ai/camel: CAMEL: Finding the Scaling Law of Agents ...\\\', \\\'description\\\': \\\'🐫 CAMEL is an open-source community dedicated to finding the sc

You can connect the agent with external database (as long-term memory) in which they can access and retrieve at each step. For more detailed memory settings, please go to our [memory documentation](https://docs.camel-ai.org/key_modules/memory.html).

### Miscs

- Setting the agent to its initial state.

    ```python
    agent.reset()
    ```

- Set the output language for the agent.

    ```python
    agent.set_output_language('french')
    ```

- The `ChatAgent` class offers several useful initialization options, including `model_type`, `model_config`, `memory`, `message_window_size`, `token_limit`, `output_language`, `tools`, and `response_terminators`.

Check [chat_agent.py](https://github.com/camel-ai/camel/blob/master/camel/agents/chat_agent.py) for detailed usage guidance.

## Remarks

Awesome. Now you have made your first step in creating a single agent. In the next chapter, we will explore the creation of different types agents along with the role playing features. Stay tuned 🦖🐆🐘🦒🦘🦕!